# Configuración Inicial

## Imports y librerías

In [ ]:
# ==== Core ====
from __future__ import annotations
import os, json, math, itertools, datetime as dt
from dataclasses import dataclass, asdict
from typing import List, Dict, Tuple, Optional, Iterable
import numpy as np
import pandas as pd

# ==== Logging ====
import logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s | %(levelname)s | %(message)s")
L = logging.getLogger("retail-media")

# ==== BigQuery (opcional, si vas a usarlo) ====
try:
    from google.cloud import bigquery
except Exception:
    bigquery = None


## Data Classes

In [ ]:
@dataclass
class BQConfig:
    project_id: str
    dataset: str
    location: str = "EU"
    credentials_json_path: Optional[str] = None  # si usas credenciales vía fichero

# EJEMPLO: bq = BQConfig(project_id="mi-proyecto", dataset="retail", credentials_json_path="gcp.json")
# df = load_bq_sql(bq, SQL_TRAFICO.format(project=bq.project_id, dataset=bq.dataset, lookback_weeks=12))



@dataclass
class PathsConfig: # Datos locales
    data_dir: str = "./data"
    csv_trafico: Optional[str] = "./data/trafico.csv"
    csv_tickets: Optional[str] = None
    csv_lineas: Optional[str] = None
    csv_tiendas: Optional[str] = None
    csv_zonas: Optional[str] = None
    csv_promos: Optional[str] = None
    csv_festivos: Optional[str] = None
    export_dir: str = "./export"




paths = PathsConfig(
  csv_trafico="./data/trafico.csv",
  csv_tickets="./data/tickets.csv",
  csv_lineas="./data/lineas.csv",
  csv_tiendas="./data/tiendas.csv",
  csv_zonas="./data/zonas.csv",
  csv_promos="./data/promos.csv",
  csv_festivos="./data/festivos.csv",
  export_dir="./export"
)
ensure_dirs(paths.data_dir, paths.export_dir)





@dataclass
class RulesConfig:
    lookback_weeks: int = 12
    include_sundays: bool = True
    valid_days: Tuple[str,...] = ("Mon","Tue","Wed","Thu","Fri","Sat","Sun")
    min_footfall: int = 50
    min_tickets_cat: int = 25
    min_lift: float = 1.05
    max_visibility_rank: int = 10
    crosssell_window_days: int = 0  # 0 = mismo ticket; >0 = ventana temporal
    topN: int = 5
    seasonality_split: bool = False

@dataclass
class WeightsConfig:
    awareness: Dict[str, float] = None
    conversion: Dict[str, float] = None
    crosssell: Dict[str, float] = None
    premium: Dict[str, float] = None
    penalties: Dict[str, float] = None

## Parámetros globales

In [ ]:


# ===== Ejemplos listos para editar =====

SLOT_DEFS: Dict[str, Tuple[int,int]] = {  # nombre -> (hora_ini, hora_fin)
    "08-11": (8,11), "11-14": (11,14), "14-17": (14,17), "17-20": (17,20), "20-22": (20,22),
}

ZONE_SYNONYMS: Dict[str,str] = {  # normalización de zonas intratienda
    "checkout":"checkout", "cola":"checkout", "endcap":"endcap", "cabecera":"endcap",
    "pasillo":"aisle", "góndola":"aisle", "entrada":"entrance"
}

OBJECTIVE_TO_SIGNALS: Dict[str, List[str]] = {
    "awareness": ["footfall_slot","visibility_rank","reach_unique_slot"],
    "conversion": ["tickets_cat_slot","net_sales_cat_slot","conv_rate_slot"],
    "cross-sell": ["lift_cross_sell","confidence_cross_sell","volume_anchor_cat_slot"],
    "premium": ["aov_slot","premium_share_slot","zone_premium_flag"]
}

WEIGHTS_DEFAULT = WeightsConfig(
    awareness={"footfall_slot":0.5, "visibility_rank":0.3, "reach_unique_slot":0.2},
    conversion={"tickets_cat_slot":0.45, "net_sales_cat_slot":0.35, "conv_rate_slot":0.2},
    crosssell={"lift_cross_sell":0.5, "confidence_cross_sell":0.3, "volume_anchor_cat_slot":0.2},
    premium={"aov_slot":0.6, "premium_share_slot":0.3, "zone_premium_flag":0.1},
    penalties={"stockouts":0.3, "promo_saturation":0.4, "low_coverage":0.3}
)

# Plantilla de brief (ejemplo)
EXAMPLE_BRIEFS: List[Dict] = [
    {
        "brand":"ACME",
        "category_focus":["coffee"],
        "objective":"cross-sell",
        "target_co_categories":["milk","cookies"],
        "budget_level":"medium",
        "preferred_days":["Fri","Sat"],
        "excluded_days":["Sun"],
        "preferred_slots":["17-20","20-22"],
        "excluded_store_types":["convenience"],
        "geos_priority":["Madrid","Bilbao"],
        "must_have_instore_locations":["checkout"],
        "notes":"evitar colas muy saturadas"
    }
]


### Slots Horarios

### Zonas Intratienda

# Carga de Datos

## BigQuery (o Stratio)

# Preprocesamiento

## Normalización Temporal

## Join con metadatos de tienda

## Join con zonas

# Cálculo de métricas

## Footfall

## Tickets Por Categoría

## Cross-sell

## Saturación de Promos

# Generación de Candidatos

## Grid combinaciones

# Scoring

## Funciones de scoring

## Penalizaciones

# Ranking y resultados

## Filtros por brief

## Ranking por marca

## Exportación

# Validación

## Holdout temporal

In [ ]:
# Ge

In [1]:
# Generación de candidatos

In [ ]:
### 

In [ ]:
import pandas as pd
import datetime as dt




# METER LOS DÍAS PROMO

# -------------------------------- Importación Datos --------------------------------

df_transaccional = pd.read_csv("TransaccionalDefinitivo.csv", delimiter = ",", header = 0) # para calcular la volumetria de tickets y 
df_articulos = pd.read_csv("Articulos.csv", delimiter = ",", header = 0) # Maestra de articulos con su categoría


def sacar_ubicacion_intratienda(articulo_id):
    # NECESITARÍA los planos. 
    return 

def ubicacion_cross_sell(articulo_id):
    return

def habitual(df, col, segmentacion, segmento):
    if isinstance(col, str):
        col = [col]
    df_seg = df[df[segmentacion] == segmento]
    agrupado = df_seg.groupby([segmentacion] + col)["Ticket Id"].nunique().reset_index(name="NumTickets")
    q75 = agrupado["NumTickets"].quantile(0.75)
    top_25 = agrupado[agrupado["NumTickets"] >= q75]
    if "TipoTienda" in col:   # Si es TipoTienda, cojo 2 tiendas siempre. 
        top_tiendas = df_seg.groupby("TipoTienda")["Ticket Id"].nunique().nlargest(2).index.tolist()
        top_25 = top_25[top_25["TipoTienda"].isin(top_tiendas)]
    return top_25




# -------------------------------- Segmentación Clientes --------------------------------
segmentos_edad = [("Jóvenes adultos", "18–24 años"), ("Adultos jóvenes", "25–34 años"), ("Adultos en etapa media", "35–44 años"), ("Adultos maduros", "45–54 años"), ("Prejubilados", "55–64 años"), ("Personas mayores", "65+ años")]
segmentos_sociodemo = [("Familias con Hijos", "Hogares con hijos en edad escolar o adolescentes"), ("Familias Jóvenes", "Hogares con niños pequeños o en primera infancia"), ("Familias sin Hijos", "Hogares conformados por parejas adultas sin hijos"), ("Personas que viven solas", "Hogares compuestos por una persona"), ("Retirados", "Hogares con personas retiradas")]
segmentos_ahorro = [("Sensibles al precio", "Clientes que priorizan el ahorro y buscan precios bajos"), ("Poco sensible al precio", "Clientes con menor preocupación por el precio y el ahorro"), ("Propensión a los cupones", "Clientes con alta propensión a utilizar cupones de descuento"), ("Promocionero", "Clientes que suelen aprovechar promociones y ofertas")]
segmentos_local = [("Local", "Clientes que prefieren productos de origen local o de cercanía"), ("No local", "Clientes que consumen habitualmente productos importados o no locales")]
segmentos_fidelidad = [("Fiel", "Realiza compras recurrentes y completas en Eroski"), ("Abandono", "Ha dejado de comprar en Eroski recientemente"), ("Compartido", "Distribuye su compra entre Eroski y otros supermercados"), ("Esporádico", "Compra de manera puntual, sin recurrencia ni cesta completa")]

# -------------------------------- Segmentación Ubicación --------------------------------
tipos_tiendas = ["Hipermercado 10000", "Hipermercado 6000", "Supermercado de atracción", "Supermercado de proximidad", "Gasolinera", "Autoservicio"]
ubicacion_intratienda = ["Escaparate", "Entrada", "Cabecera", "Lineal", "Isla promocional", "Zona caliente", "Caja", "Zona de impulso", "Exterior", "Pasillo central", "Final de pasillo", "Refrigerado", "Congelado", "Bodega", "Zona bebé"]


df_objetivos = pd.DataFrame([
    {
        "Objetivo": "Generar reconocimiento de marca",
        "TipoTienda": tipos_tiendas,
        "SegmentacionUsada": segmentos_fidelidad,
        "SegmentoTarget": ["Fiel", "Compartido"],
        "UbicacionIntraTienda": ["Escaparate", "Entrada"],
        "TramoHorario": "nulo",
        "Dia": "nulo"                      # SI ES NULO, LO CALCULO CON LA FUNCION habitual, metienod como parametros los q si tengo.
    },
    {
        "Objetivo": "Lanzamiento de nuevos productos",
        "TipoTienda": ["Supermercado", "Gasolinera"],
        "SegmentacionUsada": segmentos_edad,             # por ejemplo, 
        "SegmentoTarget": ["Jóvenes Adultos", "Adultos jóvenes"],
        "UbicacionIntraTienda": ,
        "TramoHorario": ["11–14", "17–21"],
        "Dia": ["Lunes", "Martes", "Miércoles"]
    },
    {
        "Objetivo": "Penetración en nuevos clientes",
        "TipoTienda": ["Gasolinera", "Supermercado"],
        "SegmentacionUsada": ["fidelidad", "edad"],
        "SegmentoTarget": ["Esporádico", "18–24 años"],
        "UbicacionIntraTienda": ["Caja", "Zona caliente"],
        "TramoHorario": ["7–11", "14–17"],
        "Dia": ["Entre semana"]
    },
    {
        "Objetivo": "Consideración de producto de cross-sell",
        "TipoTienda": ["Hipermercado", "Supermercado"],
        "SegmentacionUsada": ["afinidad_ahorro"],
        "SegmentoTarget": ["Promocionero", "Propensión a los cupones"],
        "UbicacionIntraTienda": ["Lineal", "Punto de impulso"],
        "TramoHorario": ["11–14", "17–21"],
        "Dia": ["Jueves", "Viernes"]
    },
    {
        "Objetivo": "Reforzar la relación con clientes",
        "TipoTienda": ["Hipermercado"],
        "SegmentacionUsada": ["fidelidad"],
        "SegmentoTarget": ["Fiel", "Compartido"],
        "UbicacionIntraTienda": ["Zona caliente", "Lineal"],
        "TramoHorario": ["14–17", "17–21"],
        "Dia": ["Fin de semana"]
    },
    {
        "Objetivo": "Momentos de consumo",
        "TipoTienda": ["Supermercado", "Gasolinera"],
        "SegmentacionUsada": ["edad", "sociodemo"],
        "SegmentoTarget": ["35–44 años", "Familias sin Hijos"],
        "UbicacionIntraTienda": ["Punto de impulso"],
        "TramoHorario": ["7–11", "14–17"],
        "Dia": ["Todos"]
    },
    {
        "Objetivo": "Alcanzar cuota en tiendas infracuota",
        "TipoTienda": ["Supermercado"],
        "SegmentacionUsada": ["local"],
        "SegmentoTarget": ["No local"],
        "UbicacionIntraTienda": ["Exterior", "Escaparate"],
        "TramoHorario": ["Todo el día"],
        "Dia": ["Entre semana"]
    },
    {
        "Objetivo": "Refuerzo promocional",
        "TipoTienda": ["Hipermercado", "Supermercado"],
        "SegmentacionUsada": ["afinidad_ahorro"],
        "SegmentoTarget": ["Sensibles al precio"],
        "UbicacionIntraTienda": ["Cabecera", "Caja"],
        "TramoHorario": ["11–14", "17–21"],
        "Dia": ["Miércoles", "Viernes"]
    },
    {
        "Objetivo": "Impulso en caja",
        "TipoTienda": ["Gasolinera", "Supermercado"],
        "SegmentacionUsada": ["sociodemo", "fidelidad"],
        "SegmentoTarget": ["Personas que viven solas", "Esporádico"],
        "UbicacionIntraTienda": ["Caja"],
        "TramoHorario": ["7–11", "14–17", "17–21"],
        "Dia": ["Todos"]
    }
])


# -------------------------------- Funciones necesarias --------------------------------
for i in num_campanias:
    brief_campania_{sheet_name} = pd.read_excel("Brief Marcas.xlsx", header = 0, sheet_name=i)


def limpiar_brief(objetivo,marca):
    if marca == ""
        brief_limpio = get.info.brief()    
    variables_null = brief_limpio.is_null() # array de variables a rellenar cuando la marca no las ha especificado. Estas variables se especifican 
    # si la variable es null, entonces la saco de un diccionario 
    # brief_limpio debe incluir
    return brief_limpio

def calcular_presencia(segmentacion):
    df = df.groupby(["Segmentacion"])["Ticket Id"].nunique()

    return # Devolver Tipo de tienda, tramo horario y dia de la semana



# -------------------------------- Generación del df combs --------------------------------
tramos_horarios = [("6.00-8.59", "Mañana"), "9.00-11.59", "12.00-13.59", "14.00-16.59", "17.00-18.59", "19.00-20.59", "21.00-23.59"]
pantallas_tienda_inc = [(CodigoCategoria//CodigoPantalla , CodigoLocalizacion_Inc, TipoTienda)] # Explicacion: CodigoLocalizacion de tiendas con cont promocional + respectivo id.//Codigo categoria. Si va a nivel pasillo o a nivel pantalla
pantallas_tienda_act = [(CodigoCategoria//CodigoPantalla , CodigoLocalizacion_Act, TipoTienda)] # Explicacion: CodigoLocalizacion de tiendas con cont promocional + respectivo id.//Codigo categoria. Si va a nivel pasillo o a nivel pantalla
num_dias_pantallas = 1000
dia_inicio = dt.date(2025, 8, 5)  # ejemplo: martes 5 de agosto de 2025
num_dias_pantallas = 5  # ejemplo
nombres_dias = ["Lunes", "Martes", "Miércoles", "Jueves", "Viernes", "Sábado", "Domingo"]
dias_promo = [(dia_inicio + dt.timedelta(days=i), nombres_dias[(dia_inicio.weekday() + i) % 7]) for i in range(num_dias_pantallas)]

combs = # hacer una combinación de cada uno. 







In [ ]:
for marca in df_marcas["Marca"].unique():
    for contenido in df_marcas[Marca == marca]["Contenido"].unique() # para cada contenido de esa marca en especifico. 
        comb_0 = combs.copy()
        requisitos_marca = brief_limpio["Marca"].values() # brief_limpio es un diccionario// data frame donde se especifica la info de asignacion para cada marca
        comb_0 = comb_0[comb_0["TipoTienda"].is_in(requisitos_marca["TipoTienda"])] # filtrado por tipo de tienda 
        comb

In [ ]:
# Calcular diferentes combinaciones, e ir jugando con lo que decia Isabel
